### MAG things
- Do taxonomy using sourmash, probably just do it for all of them, or copy tax file
- dereplicate using drep
- Create signatures for the ones that don't have a signature yet. 
- Get the quality stats for them somewhere local
- do a fastmultigather for these against high quality metagenomes. 

Maybe even take only MAGs from the 'good' metagenomes, as did for vOTUs

In [ ]:
# use the 339 'good' datasets for now. 
cp atlas/*/genomes/genomes/*.fasta ./genomes/

# concat all tax
mamba activate csvtk
csvtk concat atlas/atlas*/genomes/taxonomy/gtdb_taxonomy.tsv -t > 240705_genome_taxonomy.tsv

# concat quality info
mamba activate csvtk
csvtk concat atlas/atlas*/genomes/genome_quality.tsv -t > 240705_genome_quality.tsv

In [ ]:
# deduplicate using drep 
# use external info from checkm: https://drep.readthedocs.io/en/master/advanced_use.html#using-external-genome-quality-information
# make a completeness sheet for drep https://github.com/MrOlm/drep/issues/220
awk -F'\t' 'BEGIN {OFS=","} {print $1, $2, $3}' quality_report.tsv > new_file_name.csv
# In the new file convert headings to: genome,completeness,contamination

# use drep (retains 831 genomes)
srun --account=ctbrowngrp -p bmm -J drep -t 6:00:00 -c 32 --mem=50gb --pty bash

mamba activate drep
dRep dereplicate \
drep.999 \
--genomeInfo drepqual.csv \
-p 32 \
-g ./genomes/*.fasta \
-pa 0.9 -sa 0.999 -nc 0.30 -cm larger \
-comp 50 -con 10 -l 1000 
